In [13]:
import numpy as np
import scipy.linalg as la
import time
import bezout_5 as bz

#TEX_DIR = '/home/jp/Documents/Bezout/bezout/tex/txt'
TEX_DIR = '../tex/txt'

deg = [1]*5

with open(TEX_DIR+'/deg.txt', 'w') as f:
    f.write(str(deg))
    
t = 4
m = 16000
n = len(deg)

Field = QQ
R = PolynomialRing(Field, 'x', n)
x = R.gens()
xx = [x[0]**0] + list(x)

fshape = [d+1 for d in deg]
dx = [(i+1)*deg[i] for i in range(n)]
dy = [(n-i)*deg[i] for i in range(n)]
fn, Dx, Dy = factorial(n), prod(dx), prod(dy)
with open(TEX_DIR+'/Dx.txt', 'w') as f:
    f.write("{0:d}".format(Dx))


P = [bz.rand_poly(n-1, m, deg, t, x) for i in range(n)] + xx
#P = load('P.sobj')

save(P, 'P')
bz.P2txt(n, deg, P, TEX_DIR)
F = [bz.poly2prism(fshape, p) for p in P]

TEX_DIR

t = time.clock()
Gx, Gy, Hx, Hy = bz._GH(n, fn, deg, dx, dy)
H, K = bz._HK(n, Hx, Hy)
J = bz._J(Dx, Dy, F, n, fshape, dx, dy, Gx, Gy)
C = bz._C(n, Dx, Dy, J)
B = bz._B(n, Dx, Dy, H, K, C)

construction_B_time = time.clock() - t
with open(TEX_DIR+'/construction_B_time.txt', 'w') as f:
    f.write("{0:.4f}".format(construction_B_time))

#B = bz.block_triang(n, Dx, Dy, dx, dy, deg, B)
BB = []
for k in range(n+1):
        Bk = matrix(Field, B[k])
        BB.append(Bk[:, :])
bz.bz2txt(n, TEX_DIR, BB)

"""
computing rank of B0
"""
t = time.clock()
r0 = rank(BB[0])
rank_B0_time = time.clock() - t

with open(TEX_DIR+'/rank_B0.txt', 'w') as f:
    f.write("{0:d}".format(r0))
with open(TEX_DIR+'/rank_B0_time.txt', 'w') as f:
    f.write("{0:.4f}".format(rank_B0_time))
print("sage_rank = {0:d}".format(r0))

bb = []
for k in range(n+1):
    bb.append(np.array(BB[k], dtype=float))

b0 = bb[0]
numpy_rank = np.linalg.matrix_rank(b0)
print("numpy_rank = {0:d}".format(numpy_rank))

sage_rank = 94
numpy_rank = 94


In [14]:
"""
reduction of Bezoutian matrices
"""
epsi = abs(b0).max()/1.0e6

t = time.clock()
nb_relations = 1
while nb_relations > 0:
    bb, r0, nb_relations = bz.iteration(bb, r0, epsi)
    print bb[0].shape, r0, nb_relations

print "bbt"
bbt = []
for k in range(n+1):
    bbt.append(bb[k].T)
nb_relations = 1
while nb_relations > 0:
    bbt, r0, nb_relations = bz.iteration(bbt, r0, epsi)
    print bbt[0].shape, r0, nb_relations
reductions_time = time.clock() - t
with open(TEX_DIR+'/bezout_dim.txt', 'w') as f:
    f.write("{0:d}".format(r0))
with open(TEX_DIR+'/reductions_time.txt', 'w') as f:
    f.write("{0:.4f}".format(reductions_time))

"""
Numerical compation of the roots
"""
t = time.clock()
bbt_roots = bz.bbt2roots(bbt, epsi, r0)
compute_roots_time = time.clock() - t
with open(TEX_DIR+'/compute_roots_time.txt', 'w') as f:
    f.write("{0:.4f}".format(compute_roots_time))

test_bbtr = bz.roots_test(P, x, bbt_roots)
test_roots = np.sort(test_bbtr[:])

np.savetxt(TEX_DIR+'/test_roots.txt', test_roots,  fmt='%1.3e')

hist, bin_edges = np.histogram(np.log10(test_roots), bins='scott')
hist, bin_edges

(120, 106) 92 14
(120, 104) 92 2
(120, 104) 92 0
bbt
(104, 120) 92 0


(array([25, 33, 17,  9,  5,  1,  2]),
 array([-14.6363239 , -13.8324938 , -13.02866371, -12.22483361,
        -11.42100351, -10.61717342,  -9.81334332,  -9.00951323]))

In [15]:
with open(TEX_DIR+'/histogram.txt', 'w') as f:
    for k in range(len(hist)):
        nb_roots = hist[k]
        left_bin, right_bin = bin_edges[k], bin_edges[k+1]
        if nb_roots > 0:
            if k < len(hist)-1:
                f.write("$[{0:2.1f}, {1:2.1f}]$ & ${2:d}$\\\\\n".format(left_bin, right_bin, nb_roots))
            else:
                f.write("$[{0:2.1f}, {1:2.1f}]$ & ${2:d}$\n".format(left_bin, right_bin, nb_roots))

In [16]:
t = time.clock()
GB, grobner_dim = bz.compute_grobner(R, P, n)
print grobner_dim
grobner_time = time.clock() - t
bz.gb2txt(TEX_DIR, GB)

with open(TEX_DIR+'/grobner_dim.txt', 'w') as f:
    f.write("{0}".format(grobner_dim))

with open(TEX_DIR+'/grobner_time.txt', 'w') as f:
    f.write("{0:.4f}".format(grobner_time))

92


In [17]:
with open(TEX_DIR+'/bezout_exact.txt', 'w') as f:
    X = matrix(ZZ, Dx, 0)
    X_ortho = X.kernel().basis_matrix().LLL().transpose()
    Y, Y_time = bz.BB2Y(BB, X_ortho)
    f.write("nb de Y-relations = {0:d}\n".format(Y.nrows()))

    
    K = bz.Y2K(BB[0], X_ortho, Y)
    X_dim = Dx - K.nrows()
    f.write("X_dim = {0:d}\n".format(X_dim))

    
    Y_ortho = Y.right_kernel_matrix().LLL()
    X, X_time = bz.BB2X(BB, Y_ortho)
    N = bz.X2N(BB[0], Y_ortho, X)
    X_ortho = X.kernel().basis_matrix().LLL().transpose()
    f.write("nb de X-relations = {0:d}\n".format(X.ncols()))

In [18]:
def find_Y(BB, B0Y):
    nr, nc = BB[0].nrows(), BB[0].ncols()
    while True:
        old_nbr = B0Y.nrows()
        print old_nbr
        K_lambda = B0Y.kernel().basis_matrix()
        K = K_lambda[:, 0:nc]
        Y = matrix(Field, 0, nc)
        for k in range(1, n+1):
            Yk = K*BB[k]
            Y = block_matrix(2, 1, [Y, Yk]).LLL()
            nzr = bz.nzrows(Y) 
            Y = Y[nzr, :]
        B0Y = block_matrix(2, 1, [BB[0], Y])
        if B0Y.nrows() == old_nbr:
            break
            
    return Y

def P2field(p):
    coeffs, monoms = p.coefficients(), p.monomials()
    return sum([Field(coeffs[i])*monoms[i] for i in range(len(coeffs))])

In [19]:
B0Y = BB[0]
Y = find_Y(BB, B0Y)

BBt = []
for k in range(n+1):
    BBt.append(BB[k].transpose())
B0Yt = B0Y.transpose()
X = find_Y(BBt, B0Yt)

print "-"*10
Y_ortho = Y.right_kernel_matrix()
bb0y = BB[0]*Y_ortho.transpose()
print rank(bb0y)
X_ortho = X.right_kernel_matrix()
xbb0 = X_ortho*BB[0]
print rank(xbb0)
xbb0y = X_ortho*BB[0]*Y_ortho.transpose()
print rank(xbb0y)


120
134
136
120
134
136
----------
92
92
92


In [20]:
XBBY = []
for k in range(n+1):
    XBBY.append(X_ortho*BB[k]*Y_ortho.transpose())
bezout_exact_dim = rank(XBBY[0])
#f.write("bezout_exact_dim = {0:d}\n".format(bezout_exact_dim))
print("bezout_exact_dim = {0:d}".format(bezout_exact_dim))

bezout_exact_dim = 92


In [21]:
K_ortho = XBBY[0].kernel().basis_matrix().right_kernel_matrix()
N_ortho = XBBY[0].right_kernel_matrix().right_kernel_matrix()
#N_ortho = N.kernel().basis_matrix().transpose()

KBBN = []
for k in range(n+1):
    KBBN.append(K_ortho*XBBY[k]*N_ortho.transpose())

In [22]:
Field = GF(next_prime(200))

BBf = []
for k in range(n+1):
        Bk = matrix(Field, KBBN[k])
        BBf.append(Bk[:, :])
        
if rank(BBf[0]) == bezout_exact_dim:
    XX = []
    for k in range(n):
        xx = BBf[0].solve_right(BBf[k+1])
        XX.append(xx)
    Pf = [P2field(p) for p in P]
    test_XX = [bz.X2p(XX, Field, p) for p in Pf[:n]]
    #f.write("test_XX = {0:s}".format(test_XX))
    print("test_XX = {0:s}".format(test_XX))
else:
        print "rank deficiency !"

test_XX = [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [23]:
bb = []
for k in range(n+1):
    bb.append(np.array(KBBN[k], dtype=float))
roots = bz.BB2roots(bb)
test_roots = bz.roots_test(P, x, roots)
test_roots

array([  1.04447006e-08,   1.11130083e-08,   1.21778524e-08,
         1.14773905e-09,   3.25387274e-10,   1.03402708e-08,
         1.57032665e-09,   8.80291761e-10,   4.31204107e-10,
         7.22997289e-10,   1.83710122e-10,   6.83079561e-10,
         2.30212093e-10,   2.35890394e-10,   4.47782694e-11,
         7.00316042e-11,   1.15777035e-07,   1.96195750e-10,
         1.19633205e-09,   9.20696585e-10,   2.43815591e-09,
         4.15937867e-10,   1.51603721e-09,   1.98542095e-09,
         4.28967453e-10,   2.43701442e-11,   2.54939684e-10,
         7.65528101e-11,   3.02568809e-10,   1.35129953e-10,
         1.30115097e-09,   1.41971866e-10,   5.56761672e-11,
         2.34194705e-10,   1.43314097e-10,   6.29422775e-11,
         5.55530434e-10,   5.91428516e-11,   1.38567354e-10,
         2.85975057e-11,   1.54302808e-10,   5.12358496e-11,
         6.14025438e-11,   7.53797695e-10,   1.06325157e-09,
         3.51419892e-08,   1.26068084e-10,   8.60138265e-11,
         1.25936267e-09,

In [24]:
hist, bin_edges = np.histogram(np.log10(test_roots), bins='scott')
hist, bin_edges

(array([11, 32, 17, 24,  2,  4,  2]),
 array([-10.70256686, -10.16453982,  -9.62651278,  -9.08848574,
         -8.5504587 ,  -8.01243166,  -7.47440462,  -6.93637758]))